In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [2]:
from config import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)

In [4]:
today = datetime.datetime.today().date() # or datetime.now to use local timezone
daystart=pd.Timestamp(year=today.year, month=today.month, day=today.day, hour=0,tz='America/New_York').isoformat()

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        
        dataframe['emaclose'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['emapulse'] = ta.RSI(dataframe['emaclose'], timeperiod=3)
        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'37138.565'

In [7]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D

In [8]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2021-01-29'

In [9]:
currenthour=time.localtime().tm_hour
currentminute=time.localtime().tm_min
currentsecond=time.localtime().tm_sec
currenttime=currenthour*100+currentminute
currenttimestamp=api.get_clock().timestamp
currenttime

1047

In [10]:
api.polygon.last_quote('GME')

Quote({   'askexchange': 19,
    'askprice': 294.98,
    'asksize': 2,
    'bidexchange': 8,
    'bidprice': 293.5,
    'bidsize': 1,
    'timestamp': 1611946039695})

In [12]:
#Limit sell all within a 
trade1=True
trade2=False
trade3=True
buyordfillstatus=1
buyordcancstatus=1
buyord_avg_price=None
boughtprice=None

sellordfillstatus=1
sellordcancstatus=1
sellord_avg_price=None
soldprice=310

while trade3==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currentsecond=time.localtime().tm_sec
    currenttime=currenthour*100+currentminute
    currenttimestamp=api.get_clock().timestamp
    
    if currenttime >=600 and currenttime <=1300: 
        try:
            assetlist=api.list_positions()
        except:
            print('http error')
        for asset in assetlist:
                #if asset is in list of positions AND is more than 2.5% loss
                # sell and turn that symbol to null
            
                if asset.symbol=='GME':
                    symbol=asset.symbol                    
                    quoten=api.polygon.last_quote(symbol)
                    currentprice=quoten.askprice
                    
                    #update buy order status
                    if buyordfillstatus == None or buyordcancstatus== None:
                        buyordfillstatus=api.get_order(buyid).filled_at
                        buyordcancstatus=api.get_order(buyid).canceled_at
                    #update SELL order status
                    if sellordfillstatus == None or sellordcancstatus== None:
                        sellordfillstatus=api.get_order(sellid).filled_at
                        sellordcancstatus=api.get_order(sellid).canceled_at
                    
                    #if buy order is cancelled, buy order goes
                    if buyordcancstatus ==1:
                        pass
                    elif buyordcancstatus !=None:
                        trade1=True
                    
                    #if buy order is filled, sell order goes
                    if buyordfillstatus ==1:
                        pass 
                    elif buyordfillstatus !=None:
                        trade2=True
                        buyordavg_price=api.get_order(buyid).filled_avg_price
                        boughtprice=float(buyordavg_price)
                        trade1=False
                    
                    #if sell order is cancelled, sell order goes
                    if sellordcancstatus ==1:
                        pass
                    elif sellordcancstatus !=None:
                        trade2=True
                    
                    
                    #if sell order goes, buy order goes
                    if sellordfillstatus ==1:
                        pass 
                    elif sellordfillstatus !=None:
                        trade1=True
                        sellordavg_price=api.get_order(buyid).filled_avg_price
                        soldprice=float(buyordavg_price)
                        trade2=False
                    
                    if trade1==True and currentprice <= soldprice-5:
                        #openPo/sition = api.get_position(asset.symbol)
                        #asksize,bidsize,avgbidprice,avgaskprice=quotelister(quotes)
                        returned = api.submit_order('GME',int(1),"buy","trailing_stop","gtc",trail_price=3) # Market order to fully close position
                        print('buy '+symbol)
                        trade1=False
                        buyid=returned.id
                        time.sleep(3)
                        
                        buyordfillstatus=api.get_order(buyid).filled_at
                        buyordcancstatus=api.get_order(buyid).canceled_at
                        buyordavg_price=api.get_order(buyid).filled_avg_price
                        
                        
                    if trade2==True: 
                        if currentprice >= boughtprice+10:
                            #openPo/sition = api.get_position(asset.symbol)
                            #asksize,bidsize,avgbidprice,avgaskprice=quotelister(quotes)
                            returned1 = api.submit_order('GME',int(1),"sell","trailing_stop","gtc",trail_price=5) # Market order to fully close position
                            print('Sell 2'+symbol)
                            trade2=False
                            sellid=returned.id
                            time.sleep(3)

                            sellordfillstatus=api.get_order(sellid).filled_at
                            sellordcancstatus=api.get_order(sellid).canceled_at
                            sellordavg_price=api.get_order(sellid).filled_avg_price
                            
            

    time.sleep(2)
if currenttime >=1030:
    trade3=False



Sell GME
Sell 2GME


sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying htt

Sell GME


sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying htt

sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying htt

sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/a129235a-6e53-4b2d-9837-a8fa32c80817 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying https://api.alpaca.markets/v2/orders/7dca8bd3-cd61-4873-b12b-e3582eb11602 3 more time(s)...
sleep 3 seconds and retrying htt

KeyboardInterrupt: 

In [ ]:
api.get_order(buyid).cancelled_at

In [ ]:
trade2

In [ ]:
api.get_order('a8ff7377-4029-4b78-9b98-408b3aeb53d3')